In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from google.colab import files
import io

# PSNR calculation function
def calculate_psnr(original, inpainted):
    return cv2.PSNR(original, inpainted)

# SSIM calculation function
def calculate_ssim(original, inpainted):
    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    inpainted_gray = cv2.cvtColor(inpainted, cv2.COLOR_BGR2GRAY)
    return ssim(original_gray, inpainted_gray, data_range=255)

# MSE calculation function
def calculate_mse(original, inpainted):
    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    inpainted_gray = cv2.cvtColor(inpainted, cv2.COLOR_BGR2GRAY)
    return mean_squared_error(original_gray.flatten(), inpainted_gray.flatten())

# Calculating accuracy in the masked area
def calculate_masked_accuracy(original, inpainted, mask):
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY) / 255.0  # Normalize the mask (between 0 and 1)
    masked_original = original * mask[..., np.newaxis]
    masked_inpainted = inpainted * mask[..., np.newaxis]

    masked_mse = mean_squared_error(masked_original.flatten(), masked_inpainted.flatten())

    min_dim = min(masked_original.shape[:2])
    win_size = min(7, min_dim) if min_dim >= 7 else min_dim

    masked_ssim = ssim(
        masked_original,
        masked_inpainted,
        multichannel=True,
        win_size=win_size,
        channel_axis=2,  # Because it has 2 channels due to the mask
        data_range=1.0
    )
    return masked_mse, masked_ssim

# Resizing images to the same size due to potential padding in the mask
def resize_images_to_same_size(image1, image2):
    height, width = image1.shape[:2]
    resized_image2 = cv2.resize(image2, (width, height), interpolation=cv2.INTER_LINEAR)
    return image1, resized_image2

# Visualization
def visualize_results(original, inpainted, mask, psnr_value, ssim_value, mse_value, masked_mse, masked_ssim):
    diff = cv2.absdiff(original, inpainted)

    original_norm = (original / 255.0).astype(np.float32)
    inpainted_norm = (inpainted / 255.0).astype(np.float32)
    diff_norm = (diff / 255.0).astype(np.float32)
    mask_norm = (mask / 255.0).astype(np.float32)

    plt.figure(figsize=(18, 12))

    plt.subplot(3, 3, 1)
    plt.imshow(cv2.cvtColor(original_norm, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(3, 3, 2)
    plt.imshow(cv2.cvtColor(inpainted_norm, cv2.COLOR_BGR2RGB))
    plt.title("Inpainted Image")
    plt.axis("off")

    plt.subplot(3, 3, 3)
    plt.imshow(mask_norm, cmap='gray')
    plt.title("Mask")
    plt.axis("off")

    plt.subplot(3, 3, 4)
    plt.imshow(diff_norm, cmap='gray')
    plt.title("Difference (Abs)")
    plt.axis("off")

    plt.subplot(3, 3, 5)
    plt.text(0.1, 0.6, f"PSNR: {psnr_value:.2f}", fontsize=14)
    plt.text(0.1, 0.5, f"SSIM: {ssim_value:.4f}", fontsize=14)
    plt.text(0.1, 0.4, f"MSE: {mse_value:.2f}", fontsize=14)
    plt.text(0.1, 0.3, f"Masked MSE: {masked_mse:.2f}", fontsize=14)
    plt.text(0.1, 0.2, f"Masked SSIM: {masked_ssim:.4f}", fontsize=14)
    plt.axis("off")
    plt.title("Metrics Summary")

    # Histograms -- start
    plt.subplot(3, 3, 6)
    plt.hist(original.ravel(), bins=256, color='blue', alpha=0.7)
    plt.title("Original Histogram")

    plt.subplot(3, 3, 7)
    plt.hist(inpainted.ravel(), bins=256, color='green', alpha=0.7)
    plt.title("Inpainted Histogram")

    plt.subplot(3, 3, 8)
    plt.hist(diff.ravel(), bins=256, color='red', alpha=0.7)
    plt.title("Difference Histogram")
    # Histograms -- end

    plt.tight_layout()
    plt.show()

# Function to upload images in Colab environment
def upload_image():
    uploaded = files.upload()
    for fn in uploaded.keys():
        img = cv2.imdecode(np.frombuffer(uploaded[fn], np.uint8), cv2.IMREAD_COLOR)
        return img

# Function to run the test in Colab environment
def test_inpainting(original, inpainted, mask):
    original, inpainted = resize_images_to_same_size(original, inpainted)
    mask = cv2.resize(mask, (original.shape[1], original.shape[0]), interpolation=cv2.INTER_NEAREST)

    psnr_value = calculate_psnr(original, inpainted)
    ssim_value = calculate_ssim(original, inpainted)
    mse_value = calculate_mse(original, inpainted)
    masked_mse, masked_ssim = calculate_masked_accuracy(original, inpainted, mask)

    print(f"PSNR: {psnr_value:.2f}")
    print(f"SSIM: {ssim_value:.4f}")
    print(f"MSE: {mse_value:.2f}")
    print(f"Masked MSE: {masked_mse:.2f}")
    print(f"Masked SSIM: {masked_ssim:.4f}")

    visualize_results(original, inpainted, mask, psnr_value, ssim_value, mse_value, masked_mse, masked_ssim)

# Upload images for testing
print("Please upload the original image:")
original_image = upload_image()
print("Please upload the inpainted image:")
inpainted_image = upload_image()
print("Please upload the mask image:")
mask_image = upload_image()

# Pass images to the test function
test_inpainting(original_image, inpainted_image, mask_image)
